# Network Repair for Global Constraint

This notebook demonstrates how you can repair a network for the global constraints on the output of network. 
We use `Gurboi` to solve the Mixed-integer Quadratic Programming (MIQP). We also suggest you to use `Gurobi` as its performance is significantly faster (it also has free academic license).

This example bounds the output of Prosthesis controller (policy output) to stay below 10 [deg]. 

In [4]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow import keras
from data.data_loader import generateDataWindow
from nnreplayer.utils.options import Options
from nnreplayer.utils.utils import ConstraintsClass
from nnreplayer.repair.repair_weights_class import NNRepair


## Setup
### Load the Original Policy Model
We import a pre-trained policy. You can train a model for this dataset by running the ['00_net_train.py'] (00_net_train.py) script in this directory.

## Setup
### Load Data Set
We begin by loading the repair dataset. For this example, we need to specify the bound on the network's output in order to generate the repair samples. 
The data is provided as a numpy array.